In [1]:
import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time
import mhcflurry.data
import mhcflurry.imputation
import fancyimpute, locale

import sklearn.metrics
import sklearn.cross_validation

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5004)


In [11]:
max_ic50 = 50000
min_peptides_to_consider_allele = 10
data_dir = "../data/"

In [20]:
all_train_data = mhcflurry.data.load_allele_datasets(data_dir + "bdata.2009.mhci.public.1.txt")
#all_validation_data = mhcflurry.data.load_allele_datasets(data_dir + "bdata.2013.mhci.public.blind.1.txt")


In [8]:
imputed_train_data = mhcflurry.imputation.create_imputed_datasets(
    all_train_data,
    fancyimpute.MICE(),
    min_observations_per_peptide=2,
    min_observations_per_allele=10
)


Dropping 12235 peptides with <2 observations
Dropping 9 alleles with <10 observations: ['ELA-A1', 'HLA-B2701', 'HLA-B3508', 'HLA-B44', 'HLA-E0101', 'Mamu-B04', 'Patr-A0602', 'Patr-B0901', 'Patr-B1701']
[MICE] Completing matrix with shape (19304, 97)
[MICE] Starting imputation round 1/110, elapsed time 0.026
[MICE] Starting imputation round 2/110, elapsed time 1.362
[MICE] Starting imputation round 3/110, elapsed time 2.628
[MICE] Starting imputation round 4/110, elapsed time 3.898
[MICE] Starting imputation round 5/110, elapsed time 5.161
[MICE] Starting imputation round 6/110, elapsed time 6.430
[MICE] Starting imputation round 7/110, elapsed time 7.692
[MICE] Starting imputation round 8/110, elapsed time 8.958
[MICE] Starting imputation round 9/110, elapsed time 10.223
[MICE] Starting imputation round 10/110, elapsed time 11.490
[MICE] Starting imputation round 11/110, elapsed time 12.752
[MICE] Starting imputation round 12/110, elapsed time 14.027
[MICE] Starting imputation round 13

In [21]:
validation_df = pandas.read_csv("../data/combined_test_BLIND_dataset_from_kim2013.csv")
validation_df

,allele,peptide,length,meas,netmhc,netmhcpan,smmpmbec_cpp
0,H-2-DB,AAACNVATA,9,657.657837,154.881662,711.213514,438.530698
1,H-2-DB,AAFEFVYV,8,30831.879502,6456.542290,785.235635,10351.421667
2,H-2-DB,AAFVNDYSL,9,77.446180,17.458222,7.516229,28.054336
3,H-2-DB,AAIANQAAV,9,1.999862,9.638290,9.749896,25.703958
4,H-2-DB,AAIANQAVV,9,1.517050,8.550667,8.336812,28.773984
5,H-2-DB,AAIENYVRF,9,37.844258,252.348077,114.815362,187.068214
6,H-2-DB,AAINFITTM,9,3.155005,199.986187,389.045145,200.909281
7,H-2-DB,AAIPAPPPI,9,3243.396173,1059.253725,493.173804,295.120923
8,H-2-DB,AAKLNRPPL,9,654.636174,66.374307,77.268059,38.459178
9,H-2-DB,AALDMVDAL,9,229.614865,547.015963,597.035287,225.423921


In [52]:
alleles = sorted((
    allele for allele in all_train_data
    if len(set(all_train_data[allele].original_peptides)) >= min_peptides_to_consider_allele),
                 key=lambda allele: -1 * len(set(all_train_data[allele].original_peptides)))
alleles

print(len(alleles))
alleles

91


['HLA-A0201',
 'HLA-A0301',
 'HLA-A0203',
 'HLA-A1101',
 'HLA-A0206',
 'HLA-A3101',
 'HLA-A6802',
 'HLA-A0202',
 'HLA-A0101',
 'HLA-B0702',
 'H-2-KB',
 'H-2-DB',
 'HLA-B1501',
 'HLA-A6801',
 'HLA-A3301',
 'HLA-B2705',
 'HLA-A2601',
 'HLA-B4001',
 'HLA-B5801',
 'HLA-A2402',
 'HLA-A2902',
 'HLA-B3501',
 'HLA-B0801',
 'Mamu-A01',
 'HLA-A6901',
 'HLA-B1801',
 'HLA-A3001',
 'HLA-A2301',
 'HLA-B5701',
 'HLA-B5101',
 'HLA-B4402',
 'Mamu-B17',
 'HLA-A3002',
 'HLA-B4601',
 'Mamu-A11',
 'HLA-A0219',
 'HLA-A2403',
 'HLA-A0212',
 'Mamu-B03',
 'Mamu-B08',
 'HLA-A0211',
 'HLA-B5401',
 'HLA-B5301',
 'Mamu-A02',
 'HLA-B4403',
 'HLA-A0216',
 'HLA-B4501',
 'HLA-B3901',
 'HLA-B4002',
 'HLA-B4801',
 'HLA-B1517',
 'Mamu-B01',
 'HLA-A8001',
 'Patr-B0101',
 'HLA-A3201',
 'Patr-A0901',
 'Patr-A0701',
 'HLA-A2501',
 'HLA-B0802',
 'Mamu-A2201',
 'H-2-KD',
 'HLA-B2703',
 'HLA-B1503',
 'Patr-A0101',
 'H-2-KK',
 'HLA-B1509',
 'Patr-B2401',
 'Patr-A0301',
 'H-2-DD',
 'H-2-LD',
 'Patr-A0401',
 'HLA-B0803',
 'HLA-A26

In [54]:
dropout_probabilities = [0.0, 0.5]
embedding_output_dims_and_layer_sizes_list = [(32, [64]), (8, [4])]
activations = ["tanh"]

models_params_list = []

for impute in [False, True]:
    for dropout_probability in dropout_probabilities:
        for (embedding_output_dim, layer_sizes) in embedding_output_dims_and_layer_sizes_list:
            for activation in activations:
                models_params_list.append(dict(
                    impute=impute,
                    dropout_probability=dropout_probability,  
                    embedding_output_dim=embedding_output_dim,
                    layer_sizes=layer_sizes,
                    activation=activation))

print("%d models" % len(models_params_list))
models_params_explored = set.union(*[set(x) for x in models_params_list])
models_params_explored


8 models


{'activation',
 'dropout_probability',
 'embedding_output_dim',
 'impute',
 'layer_sizes'}

In [22]:
def log_to_ic50(log_value):
        """
        Convert neural network output to IC50 values between 0.0 and
        self.max_ic50 (typically 5000, 20000 or 50000)
        """
        return max_ic50 ** (1.0 - log_value)

def make_scores(ic50_y, ic50_y_pred, sample_weight=None, threshold_nm=500):     
    y_pred = mhcflurry.common.ic50_to_regression_target(ic50_y_pred, max_ic50)
    try:
        auc = sklearn.metrics.roc_auc_score(ic50_y <= threshold_nm, y_pred, sample_weight=sample_weight)
    except ValueError:
        auc = numpy.nan
    try:
        f1 = sklearn.metrics.f1_score(ic50_y <= threshold_nm, ic50_y_pred <= threshold_nm, sample_weight=sample_weight)
    except ValueError:
        f1 = numpy.nan
    try:
        tau = scipy.stats.kendalltau(ic50_y_pred, ic50_y)[0]
    except ValueError:
        tau = numpy.nan
    
    return dict(
        auc=auc,
        f1=f1,
        tau=tau,
    )    

def mean_with_std(grouped_column, decimals=3):
    pattern = "%%0.%df" % decimals
    return pandas.Series([
        (pattern + " +/ " + pattern) % (m, s) if not pandas.isnull(s) else pattern % m
        for (m, s) in zip(grouped_column.mean(), grouped_column.std())
    ], index = grouped_column.mean().index)

def allele_data_to_df(data):
    d = data._asdict()
    d["X_index"] = [x for x in d["X_index"]]
    d["X_binary"] = [x for x in d["X_binary"]]
    df = pandas.DataFrame(d).set_index('peptides')
    return df

def make_2d_array(thing):
    return numpy.array([list(x) for x in thing])

def df_to_allele_data(df):
    d = dict((col, df[col].values) for col in df)
    d["X_index"] = make_2d_array(d["X_index"])
    (d["max_ic50"],) = list(df.max_ic50.unique())
    return mhcflurry.data.AlleleData(peptides = df.index.values, **d)


In [45]:
a=all_train_data["Mamu-A02"]

In [49]:
len(a.Y)

6027

In [ ]:
# train and test models, adding columns to validation_df_with_mhcflurry
validation_df_with_mhcflurry = validation_df.copy()
def make_and_fit_model(allele, original_params):
    params = dict(original_params)
    impute = params["impute"]
    del params["impute"]
    model = mhcflurry.Class1BindingPredictor.from_hyperparameters(max_ic50=max_ic50, **params)
    print("Fitting model for allele %s (%d + %d): %s" % (
            allele, len(all_train_data[allele].Y), len(imputed_train_data[allele].Y), str(original_params)))
    t = -time.time()
    model.fit(all_train_data[allele].X_index,
              all_train_data[allele].Y,
              sample_weights=all_train_data[allele].weights,
              X_pretrain=imputed_train_data[allele].X_index if impute else None,
              Y_pretrain=imputed_train_data[allele].Y if impute else None,
              sample_weights_pretrain=imputed_train_data[allele].weights if impute else None,
              verbose=False,
              n_training_epochs=250)
    t += time.time()
    print("Trained in %d sec" % t)
    return model

models_and_scores = {}
for (i, allele) in enumerate(alleles[:2]):
    if allele not in validation_df_with_mhcflurry.allele.unique():
        print("Skipping allele %s: not in test set" % allele)
        continue
    values_for_allele = []
    for (j, params) in enumerate(models_params_list):
        print("Allele %d model %d" % (i, j))
        model = make_and_fit_model(allele, params)
        predictions = model.predict_peptides_ic50(
            list(validation_df_with_mhcflurry.ix[validation_df_with_mhcflurry.allele == allele].peptide))
        print("test set size: %d" % len(predictions))
        validation_df_with_mhcflurry.loc[(validation_df_with_mhcflurry.allele == allele),
                                         ("mhcflurry %d" % j)] = predictions
        scores = make_scores(validation_df_with_mhcflurry.ix[validation_df.allele == allele].meas,
                            predictions)
        print(scores)
        values_for_allele.append((params, model, scores))
        
    models_and_scores[allele] = values_for_allele
        
len(models)

Allele 0 model 0
Fitting model for allele HLA-A0201 (32876 + 80965): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 32, 'dropout_probability': 0.0, 'layer_sizes': [64]}
Trained in 126 sec
test set size: 2126
{'tau': 0.48932780465284226, 'auc': 0.85313512839690842, 'f1': 0.79509632224168136}
Allele 0 model 1
Fitting model for allele HLA-A0201 (32876 + 80965): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 8, 'dropout_probability': 0.0, 'layer_sizes': [4]}
Trained in 121 sec
test set size: 2126
{'tau': 0.61646556927615426, 'auc': 0.92651102325747048, 'f1': 0.87870855148342064}
Allele 0 model 2
Fitting model for allele HLA-A0201 (32876 + 80965): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 32, 'dropout_probability': 0.5, 'layer_sizes': [64]}
Trained in 143 sec
test set size: 2126
{'tau': 0.6263944343382003, 'auc': 0.92696735762367766, 'f1': 0.87851275399913542}
Allele 0 model 3
Fitting model for allele HLA-A0201 (32876 + 80965): {'activ

In [35]:
validation_df_with_mhcflurry_results = validation_df_with_mhcflurry.ix[validation_df_with_mhcflurry.allele.isin(models_and_scores)]
validation_df_with_mhcflurry_results

,allele,peptide,length,meas,netmhc,netmhcpan,smmpmbec_cpp,mhcflurry 0,mhcflurry 1,mhcflurry 2,mhcflurry 3,mhcflurry 4,mhcflurry 5,mhcflurry 6,mhcflurry 7
27292,Mamu-A02,AAFKVRPTF,9,20.323570,17.298164,102.093948,66.680677,1929.528011,87.088801,11.467875,9.419386,35.338602,42.833066,100.443275,130.157508
27293,Mamu-A02,ADILLHSTYF,10,23280.912577,352.370871,17021.585084,63.386971,114.218985,52.920956,49.660647,37.058487,149.710778,117.707672,371.824785,497.202884
27294,Mamu-A02,ADTLLHSTYF,10,34.593938,281.190083,21183.611352,38.547836,13.309919,42.700424,13.384578,16.681969,40.812935,38.096691,111.726009,151.347783
27295,Mamu-A02,AEFKYIAAV,9,78342.964277,18836.490895,24210.290467,128233.058266,22836.132699,38485.040261,6578.406981,10565.490750,7175.212366,6950.027649,3426.579617,4203.947164
27296,Mamu-A02,ANSDLGTWQM,10,7961.593504,314.050869,5610.479760,69.823240,293.792671,6235.001258,273.883607,77.412366,773.527466,690.675956,565.568296,837.066494
27297,Mamu-A02,ANSIETIVLM,10,76.736149,1527.566058,3881.503660,24.717241,555.781712,197.307487,66.940533,408.630177,435.411187,414.580411,380.241365,592.528137
27298,Mamu-A02,APPGYALL,8,17139.573075,23604.782332,23227.367964,25527.013027,43744.991942,26260.703147,34330.147270,3036.138006,18123.182283,15718.532178,13036.900827,13536.007545
27299,Mamu-A02,APRTLVYLL,9,186.208714,2027.682720,22594.357702,10209.394837,21312.083452,19.982785,9.245758,169.651147,546.071671,367.968456,1535.386113,2429.870929
27300,Mamu-A02,AQFSPQYL,8,86896.042929,8511.380382,4570.881896,3104.559588,1023.554409,4308.377349,1284.966147,1220.598081,1199.063649,1317.435588,1707.669867,2735.962302
27301,Mamu-A02,AQIDNYNKF,9,78342.964277,6998.419960,1927.524913,8531.001140,3514.908804,3904.649784,1260.787317,2941.464537,1985.499830,1739.687126,1600.959717,3642.074912


In [40]:
scores_df = collections.defaultdict(list)
predictors = validation_df_with_mhcflurry_results.columns[4:]

for (allele, grouped) in validation_df_with_mhcflurry_results.groupby("allele"):
    scores_df["allele"].append(allele)
    scores_df["test_size"].append(len(grouped.meas))
    for predictor in predictors:
        scores = make_scores(grouped.meas, grouped[predictor])
        for (key, value) in scores.items():
            scores_df["%s_%s" % (predictor, key)].append(value)
    
scores_df = pandas.DataFrame(scores_df)
scores_df.sort("test_size", ascending=False, inplace=True)
scores_df.index = scores_df.allele
scores_df

/home/tim/anaconda2/envs/standard-2.7/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,allele,mhcflurry 0_auc,mhcflurry 0_f1,mhcflurry 0_tau,mhcflurry 1_auc,mhcflurry 1_f1,mhcflurry 1_tau,mhcflurry 2_auc,mhcflurry 2_f1,mhcflurry 2_tau,...,netmhc_auc,netmhc_f1,netmhc_tau,netmhcpan_auc,netmhcpan_f1,netmhcpan_tau,smmpmbec_cpp_auc,smmpmbec_cpp_f1,smmpmbec_cpp_tau,test_size
allele,,,,,,,,,,,,,,,,,,,,,
Mamu-A02,Mamu-A02,0.812289,0.627566,0.421353,0.788411,0.600583,0.411389,0.843924,0.653846,0.472038,...,0.929688,0.767123,0.598964,0.883087,0.748092,0.524622,0.810692,0.64,0.478589,388


In [42]:
scores_df.ix["Mamu-A02"]

allele              Mamu-A02
mhcflurry 0_auc     0.812289
mhcflurry 0_f1      0.627566
mhcflurry 0_tau     0.421353
mhcflurry 1_auc     0.788411
mhcflurry 1_f1      0.600583
mhcflurry 1_tau     0.411389
mhcflurry 2_auc     0.843924
mhcflurry 2_f1      0.653846
mhcflurry 2_tau     0.472038
mhcflurry 3_auc     0.862996
mhcflurry 3_f1      0.674352
mhcflurry 3_tau     0.482676
mhcflurry 4_auc     0.855605
mhcflurry 4_f1       0.65896
mhcflurry 4_tau     0.494337
mhcflurry 5_auc     0.860837
mhcflurry 5_f1      0.666667
mhcflurry 5_tau     0.502497
mhcflurry 6_auc     0.817825
mhcflurry 6_f1      0.638418
mhcflurry 6_tau     0.450735
mhcflurry 7_auc     0.816791
mhcflurry 7_f1      0.631579
mhcflurry 7_tau     0.448527
netmhc_auc          0.929688
netmhc_f1           0.767123
netmhc_tau          0.598964
netmhcpan_auc       0.883087
netmhcpan_f1        0.748092
netmhcpan_tau       0.524622
smmpmbec_cpp_auc    0.810692
smmpmbec_cpp_f1         0.64
smmpmbec_cpp_tau    0.478589
test_size     

In [ ]:
summary_df = scores_df[scores_df.allele.isin(alleles)]
summary_df